In [0]:
import pyspark

In [0]:
data=spark.sql("select * from customer_churn_csv")

In [0]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [0]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [0]:
data.dtypes

Out[33]: [('Names', 'string'),
 ('Age', 'double'),
 ('Total_Purchase', 'double'),
 ('Account_Manager', 'int'),
 ('Years', 'double'),
 ('Num_Sites', 'double'),
 ('Onboard_date', 'string'),
 ('Location', 'string'),
 ('Company', 'string'),
 ('Churn', 'int')]

In [0]:
data.columns

Out[34]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,StringIndexer,OneHotEncoder)

In [0]:
assembler=VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol='features')

In [0]:
output=assembler.transform(data)

In [0]:
final_data=output.select('features','churn')

In [0]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
log_reg_titanic=LogisticRegression(featuresCol='features',labelCol='churn')
fit_model=log_reg_titanic.fit(train_data)
train_sumary=fit_model.summary

In [0]:
train_sumary.predictions.describe().show()

+-------+-------------------+-------------------+
summary| churn| prediction|
+-------+-------------------+-------------------+
 count| 633| 633|
 mean| 0.1627172195892575|0.11532385466034756|
 stddev|0.36939937219022917| 0.3196649714980352|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
pred_and_labels=fit_model.evaluate(test_data)

In [0]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[28.0,8670.98,0.0...| 0|[7.64084953399496...|[0.99951981054728...| 0.0|
[28.0,11128.95,1....| 0|[4.04945761873266...|[0.98286683558196...| 0.0|
[28.0,11245.38,0....| 0|[3.56952242805119...|[0.97260246610543...| 0.0|
[29.0,9378.24,0.0...| 0|[4.63558143675879...|[0.99039272979888...| 0.0|
[30.0,8874.83,0.0...| 0|[3.02824275146600...|[0.95383385449913...| 0.0|
[30.0,10744.14,1....| 1|[1.53320973315624...|[0.82247545119686...| 0.0|
[30.0,12788.37,0....| 0|[2.50810213999345...|[0.92470786182965...| 0.0|
[31.0,7073.61,0.0...| 0|[2.91127252294039...|[0.94840087309746...| 0.0|
[31.0,8688.21,0.0...| 0|[6.53596574863932...|[0.99855177270496...| 0.0|
[31.0,11743.24,0....| 0|[6.61141423718872...|[0.99865687764833...| 0.0|
[32.0,8011.38,0.0...| 0|[1.87939572132982...|[0.86754170234551...| 0.0|
[32.0,8617.98,1.0...| 1|[0.86570916611037...|[0.70385208102057...| 0.0|
[33.0,4711.89,0.0...| 0|[5.67773542418177...|[0.99659036790086...| 0.0|
[33.0,5738.82,0.0...| 0|[4.03240240233906...|[0.98257725443765...| 0.0|
[33.0,7492.9,0.0,...| 0|[4.56444994656657...|[0.98969175980473...| 0.0|
[33.0,7720.61,1.0...| 0|[1.57173880935111...|[0.82803134802392...| 0.0|
[33.0,10709.39,1....| 0|[6.08391855728566...|[0.99772595521990...| 0.0|
[33.0,11370.28,1....| 0|[6.56291634899277...|[0.99859022775485...| 0.0|
[33.0,12249.96,0....| 0|[5.53179453902005...|[0.99605673256698...| 0.0|
[33.0,14160.05,1....| 0|[5.18870014753556...|[0.99445170105975...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
my_eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [0]:
AUC=my_eval.evaluate(pred_and_labels.predictions)
AUC

Out[47]: 0.8131528046421664

In [0]:
# Predict on new data

In [0]:
# final_data=spark.sql("select * from new_customers_csv")

In [0]:
final_lr_model=log_reg_titanic.fit(final_data)

In [0]:
new_customers=spark.sql("select * from new_customers_csv")

In [0]:
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [0]:
test_new_customers=assembler.transform(new_customers)

In [0]:
test_new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [0]:
final_results=final_lr_model.transform(test_new_customers)

In [0]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,1.0...|[2.22168705251434...|[0.90218018099704...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595013...|[0.00198380445829...| 1.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|[-3.7691621189411...|[0.02255110110411...| 1.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,0.0,...|[-5.0956222016513...|[0.00608622642085...| 1.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,1....|[1.10475867224171...|[0.75115067517478...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,1.0...|[-1.6896019277060...|[0.15582819767641...| 1.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+

In [0]:
final_results.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+

In [0]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
 count| 6| 6| 6| 6| 6| 6| 6| 6| 6|
 mean| null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334| null| null| null|
 stddev| null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785| null| null| null|
 min|Andrew Mccall| 22.0| 100.0| 0| 1.0| 8.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
 max| Taylor Young| 65.0| 13147.71| 1| 10.0| 15.0|2016-10-28 05:32:13|Unit 0789 Box 073...| Wood LLC|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+